In [30]:
import pandas as pd

raw_data_file = "../data/33clusters30PCs_withNames.csv"


import numpy as np

raw_data = pd.read_csv(raw_data_file, index_col=None)

In [31]:
raw_data

,code,group,diagnosis,chapter
0,F70(Mild)|F71(Moderate)|F72(Severe)|F73(Profound),10,Intellectual disabilities,1.0
1,F88_Global Developmental Delay,10,Global Developmental Delay,1.0
2,F90.8,30,Other Specified Attention-Deficit/Hyperactivit...,1.0
3,F51.01,15,Insomnia Disorder,12.0
4,F51.11,15,Hypersomnolence Disorder,12.0
...,...,...,...,...
218,F50.9,28,Unspecified Feeding or Eating Disorder,10.0
219,F98.0,20,Enuresis,11.0
220,F98.1,6,Encopresis,11.0
221,N39.498(with urinary symptoms)|R15.9(with feca...,30,Other Specified Elimination Disorder,11.0


In [32]:
 # Split pipe-separated values in 'code' into separate rows, repeating other columns,
raw_data['code'] = raw_data['code'].astype(str)
raw_data['code_list'] = raw_data['code'].str.split('|')
expanded = raw_data.explode('code_list').copy()
expanded['code'] = expanded['code_list'].str.strip()
expanded = expanded.drop(columns=['code_list']).reset_index(drop=True)
# now `expanded` has one row per code; assign back if you want to replace raw_data,
raw_data = expanded
raw_data

,code,group,diagnosis,chapter
0,F70(Mild),10,Intellectual disabilities,1.0
1,F71(Moderate),10,Intellectual disabilities,1.0
2,F72(Severe),10,Intellectual disabilities,1.0
3,F73(Profound),10,Intellectual disabilities,1.0
4,F88_Global Developmental Delay,10,Global Developmental Delay,1.0
...,...,...,...,...
551,F98.1,6,Encopresis,11.0
552,N39.498(with urinary symptoms),30,Other Specified Elimination Disorder,11.0
553,R15.9(with fecal symptoms),30,Other Specified Elimination Disorder,11.0
554,R32(with urinary symptoms),28,Unspecified Elimination Disorder,11.0


In [46]:
raw_data[np.isin(raw_data['code'].values, "F70")]

,code,group,diagnosis,chapter


In [48]:
raw_data['code']

0                           F70(Mild)
1                       F71(Moderate)
2                         F72(Severe)
3                       F73(Profound)
4      F88_Global Developmental Delay
                    ...              
551                             F98.1
552    N39.498(with urinary symptoms)
553        R15.9(with fecal symptoms)
554        R32(with urinary symptoms)
555        R15.9(with fecal symptoms)
Name: code, Length: 556, dtype: object

In [63]:
match = raw_data['code'].astype(str).str.contains('F80', na=False, regex=False)


raw_data[match]

,code,group,diagnosis,chapter,code_key
339,F80.2,32,Language Disorder,1.0,F80.2
386,F80.0,32,Speech Sound Disorder,1.0,F80.0
445,F80.81,22,Childhood-Onset Fluency Disorder (Stuttering),1.0,F80.81
489,F80.82,32,Social (Pragmatic) Communication Disorder,1.0,F80.82
508,F80.9,28,Unspecified Communication Disorder,1.0,F80.9


In [54]:
database = pd.read_csv("../data/datalong.csv", index_col=None)

In [56]:
database["code"] = database["code"].str.split('_').str[0]

In [58]:
# ...existing code...
# normalize/extract a base code from raw_data (handles "F70(Mild)" etc.)
raw_data['code_key'] = raw_data['code'].astype(str).str.extract(r'(^[A-Z]\d{2}(?:\.\d+)?)', expand=False)

# build mapping from code_key -> diagnosis (take first if duplicates)
mapping = raw_data.groupby('code_key')['diagnosis'].first().to_dict()

# map to database (database['code'] should already be cleaned like "F70" or "F60.2")
database['disorder'] = database['code'].map(mapping)

# optional: see how many didn't match
print(f"unmatched codes: {database['disorder'].isna().sum()}")
# ...existing code...

unmatched codes: 98


In [66]:
database.drop(columns=["Unnamed: 0"], inplace=True)

In [68]:
#reorganize columns
database = database[['code', 'disorder', 'symptome']]

In [70]:
database.to_csv("../data/datalong.csv")

In [60]:
database[database['disorder'].isna()]

,Unnamed: 0,code,symptome,disorder
89,89,L98.1,The skin picking causes clinically significant...,NaN
90,90,L98.1,Recurrent skin picking resulting in skin lesions,NaN
91,91,L98.1,Repeated attempts to decrease or stop skin pic...,NaN
92,92,F42,The obsessions or compulsions are time-consumi...,NaN
93,93,F42,The behaviors or mental acts are aimed at prev...,NaN
...,...,...,...,...
908,908,F80.89,Difficulties following rules for conversation ...,NaN
909,909,F80.89,Persistent difficulties in the social use of v...,NaN
910,910,F80.89,Deficits in using communication for social pur...,NaN
911,911,F80.89,Difficulties understanding what is not explici...,NaN
